In [27]:
def negator(s):
    s = s.replace("n't", " not").replace(".", " . ")
    should_negate_word = False
    text = ''
    words = s.split()
    for word in words:
        if word == 'not' or word == 'no' or word == 'n\'t':
            should_negate_word = not should_negate_word
        elif word == '.':
            should_negate_word = False
        elif should_negate_word:
            text += 'not_' + word + ' '
        else:
            text += word + ' '
    return text

def ngram(text, size):
    words = text.split()
    shingles = [words[i:i+size] for i in xrange(len(words) - size + 1)]
    shingles = ["_".join(i) for i in shingles]
    return " ".join(shingles)

In [28]:
from nltk.stem import PorterStemmer
from nltk.tokenize import SpaceTokenizer
import re
import csv

stopwords = open('stopwords.csv').readlines()[0].rstrip().split(', ')

tokenizer = SpaceTokenizer()
stemmer = PorterStemmer()

pipeline = [lambda s: re.sub('<[^>]*>', ' ', s),
            lambda s: re.sub("[^.'\w\s]", '', s),
            lambda s: re.sub('[\d]', '', s),
            lambda s: s.lower(),
            lambda s: ' '.join(filter(lambda s: not (s in stopwords), tokenizer.tokenize(s))),
            #lambda s: ' '.join(map(lambda t: stemmer.stem(t), tokenizer.tokenize(s))),
            lambda s: negator(s),
            #lambda s: ngram(s, 2)
           ]

def preprocess_text(text, pipeline=pipeline):
    if len(pipeline) == 0:
        return text
    else:
        return preprocess_text(pipeline[0](text), pipeline[1:])

In [29]:
def preprocess_review(r, test=False):
    if (test):
        SUMMARY_INDEX = 7
        TEXT_INDEX = 8  
    else:
        SUMMARY_INDEX = 8
        TEXT_INDEX = 9
    
    r[SUMMARY_INDEX] = preprocess_text(r[SUMMARY_INDEX])
    r[TEXT_INDEX] = preprocess_text(r[TEXT_INDEX])
    return r

In [30]:
import csv
import sys

reload(sys)
sys.setdefaultencoding('utf-8')

def parallel_preprocess(input_file, output_file, test=False):
    # Load csv
    reviewsRDD = sc.textFile(input_file, use_unicode=False)
    reviewsRDD = reviewsRDD.mapPartitions(lambda x: csv.reader(x))

    # Save header
    header = reviewsRDD.first()

    # Extract header and preprocess
    reviewsRDD = reviewsRDD.filter(lambda x: x != header)\
                           .map(lambda x: preprocess_review(x, test))
    ppreviews = reviewsRDD.collect()

    # Insert header
    ppreviews.insert(0, header)

    # Save csv
    with open(output_file, 'wb') as f:
        writer = csv.writer(f)
        writer.writerows(ppreviews)

In [31]:
# Add test=True to preprocess test files
#parallel_preprocess('../data/test.csv', '../data/sparktest.csv', test=True)